In [2]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
try:
    with driver.session() as session:
        result = session.run("RETURN 1 AS test")
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [4]:
import os
with open('./pathCSV.txt','r') as f:
    path = f.read()
os.path.exists(path)

True

In [14]:
business_path = "business.csv"
review_path = "review.csv"
# tip_path = "yelp_academic_dataset_tip.json"
user_path = "user.csv"

business_path, review_path, user_path = list(map(lambda y: path + '/' + y,[business_path, review_path, user_path]))
# business_path, review_path, user_path = list(map(lambda y:'"'+ path + '/' + y+'"',[business_path, review_path, user_path]))
print(business_path)

../filteredCSV/business.csv


In [33]:
def import_csv(tx, csv_file_path, query):
    with open(csv_file_path, 'r') as file:
        tx.run(query, file=file)
    # with open(csv_file_path, 'r') as file:
    #     csv_data = file.read()
    # tx.run(query,file=file)
# csv_file_path = "path/to/your/data.csv"

import pandas as pd

# Read CSV into Pandas DataFrame
df = pd.read_csv(business_path)
print(df.columns)

# Function to create nodes and relationships
def create_graph(tx, df):
    for _, row in df.iterrows():
        # Example: Create a node with properties from the CSV row
        query = """CREATE (n:Businses  {name:$name,
                    business_id:$business_id, 
                    address:$address,
                    city:$city,
                    state:$state,
                    latitude:$latitude,
                    longitude:$longitude,
                    stars:$stars,
                    review_count:$review_count,
                    hours:$hours,
                    attributes:$attributes})
                    """

        tx.run(query, business_id=row['business_id'],
                    address=['address'],
                    name=row['name'],
                    city=row['city'],
                    state=row['state'],
                    latitude=row['latitude'],
                    longitude=row['longitude'],
                    stars=row['stars'],
                    review_count=row['review_count'],
                    hours=row['hours'],
                    attributes=row['attributes'])

# Execute the function within a transaction
with driver.session() as session:
    session.execute_write(create_graph, df)

# Close the driver
driver.close()

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


C:\Users\susha\AppData\Local\Temp\ipykernel_6480\3728677617.py:45: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
